<a href="https://colab.research.google.com/github/cgsnider/Restaurant-Recs/blob/main/Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The recommender model.
Designed to be run on Colab.

Note: Requires access to your google drive and for the teams folder "ML" to be located in the root directory of your drive (cannot be inside another folder)


In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
import pandas as pd
from lightfm.data import Dataset

/Users/charles/miniconda3/envs/restaurant_recs/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [30]:
# Utiliy Functions:

def get_features(row:pd.Series):
    return [f'{label}: {value}' for label, value in row.iteritems()]

def get_ided_features(row:pd.Series):
    return (row[0], [f'{label}: {value}' for label, value in row[1:].iteritems()])


In [3]:
reviews_df = pd.read_json('data/cleaned/yelp_academic_dataset_review.json')
reviews_df

,review_id,user_id,business_id,stars
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5
9,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3
11,l3Wk_mvAog6XANIuGQ9C7Q,ZbqSHbgCjzVAqaa7NKWn5A,EQ-TZ2eeD_E0BHuvoaeG5Q,4
...,...,...,...,...
6990270,7NgXAuTFiJHYbuepOPwU0w,x1QLCwZGFAjxRRw4EHc3-g,1_BVWDzi5cVqWxNe9bOMMQ,5
6990272,wD5ZWao_vjyT2h4xmGam8Q,7L7GL5Pi2cf8mbm2Dpw4zw,e_E-jq9mwm7wk75k7Yi-Xw,5
6990274,YVX1Wsa4LYxjvFwuHBb_gA,RKPkxOYQlM0BjhM-H6_vAw,X4mouE_cMiwbfyCPZ_K-FA,4
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5


In [4]:
restaurant_df = pd.read_json('data/cleaned/yelp_academic_dataset_restaurant_reduced.json')
restaurant_df

,buisness_id,stars,review_count,postal_code,street,businessacceptscreditcards,restaurantsdelivery,restaurantspricerange2,bikeparking,outdoorseating,...,intimate,hipster,classy,trendy,casual,restaurantsgoodforgroups,alcohol,restaurantsattire,noiselevel,RestaurantsAttire
0,MTSW4McQd7CbVtyjqoe9mw,4.0,80,19107,True,False,False,1,True,False,...,False,False,False,False,True,True,free,none,average,none
1,mWMc6_wTdE0EUBKIGXDVfA,4.5,13,18054,False,True,True,2,True,False,...,False,False,False,False,True,True,no,none,average,none
2,CF33F8-E6oudUQ46HnavjQ,2.0,6,37015,False,True,True,1,False,True,...,False,False,False,False,True,True,no,casual,average,casual
3,bBDDEgkFA1Otx9Lfe7BZUQ,1.5,10,37207,False,True,True,1,False,True,...,False,False,False,False,True,False,no,casual,average,casual
4,eEOYSgkmpB90uNA7lDOMRA,4.0,10,33602,False,True,True,2,False,False,...,False,False,False,False,False,True,no,none,average,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44577,l9eLGG9ZKpLJzboZq-9LRQ,3.0,11,19018,False,True,True,1,True,False,...,False,False,False,False,True,True,no,none,average,none
44578,cM6V90ExQD6KMSU3rRB5ZA,4.0,33,83704,False,True,False,1,True,True,...,False,False,False,False,True,True,free,casual,loud,casual
44579,1jx1sfgjgVg0nM6n3p0xWA,4.5,41,85737,False,True,False,2,True,True,...,False,False,False,False,True,True,free,none,average,none
44580,WnT9NIzQgLlILjPT0kEcsQ,4.5,35,19147,True,True,True,2,True,False,...,False,False,False,False,True,True,free,casual,average,casual


In [39]:
item_features = []
restaurant_df.drop("buisness_id", axis=1).apply(lambda r : item_features.extend(get_features(r)), axis = 1)
None

In [33]:
# item_features = []
# restaurant_df.apply(lambda r : item_features.append(f"stars:{r.stars}"), axis=1)
# test = ((item_features[0][0] ,[item_features[0][1]]), (item_features[1][0] ,[item_features[1][1]]))

#Builds the mapping for restaurant's id (business_id) and customer's id (user_id)
dataset = Dataset()
dataset.fit(reviews_df.user_id.unique(), reviews_df.business_id.unique(), item_features=item_features)

In [8]:
num_users, num_items = dataset.interactions_shape()
print(f'num_users: {num_users}, num_items: {num_items}')

num_users: 1333169, num_items: 44582


In [9]:
#Runtime ~4min on x86 much faster on M1
interactions, weights = dataset.build_interactions(((review.user_id, review.business_id) for i, review in reviews_df.iterrows()))

In [35]:
item_features = []
restaurant_df.apply(lambda r : item_features.append(get_ided_features(r)), axis=1)

item_features = dataset.build_item_features(tuple(item_features))

In [36]:
item_features

<44582x47785 sparse matrix of type '<class 'numpy.float32'>'
	with 1053753 stored elements in Compressed Sparse Row format>